In [194]:
!pip install -q pyomo

Ex1) solution
max 0.65x1 + 0.75x2 − 5.85x3 − 8.35x4 − 12.38x5 + 30x6
s.t. − 0.5x1 − 3.8x3 − 0.9x4 + 0.9x5 ≤ 24.8
x1 − x4 − x6 ≤ 0.8
−2.9x2 − 1.9x3 − 0.9x6 ≤ −71.4
4.7x4 + 8.02x5 + 6.15x6 ≤ 95
−0.7x1 − 1.4x2 + 7.9x3 − 2.9x4 + 0.9x5 ≤ −47
xi ≥ 0 ∀i ∈ {1, 2, . . . , 6}.

In [195]:
from pyomo.environ import * 

In [196]:
import numpy as np

In [197]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')


In [198]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])

(6, 7)
Num rows: 6
Num cols: 7


In [199]:
print(coef)

[[  0.65   0.75  -5.85  -8.35  -2.38  30.     0.  ]
 [ -0.5    0.    -3.8   -0.9    0.9    0.    24.8 ]
 [  1.     0.     0.    -1.     0.    -1.     0.8 ]
 [  0.    -2.9   -1.9    0.     0.    -0.9  -71.4 ]
 [  0.     0.     0.     4.7    8.02   6.15  95.  ]
 [ -0.7   -1.4    7.9   -2.9    0.9    0.   -47.  ]]


In [200]:
model_lab4 = ConcreteModel()

In [201]:
N = coef.shape[1]-1    #substract last column of scalar.

In [202]:
M = coef.shape[0]-1    #substract first row 

In [203]:
obj_coef = coef[0,:-1]   #objective 
print(obj_coef)

[ 0.65  0.75 -5.85 -8.35 -2.38 30.  ]


In [204]:
constr_coef = coef[1:,:-1]    #constraints coefficent
print(constr_coef)
print(constr_coef.shape)

[[-0.5   0.   -3.8  -0.9   0.9   0.  ]
 [ 1.    0.    0.   -1.    0.   -1.  ]
 [ 0.   -2.9  -1.9   0.    0.   -0.9 ]
 [ 0.    0.    0.    4.7   8.02  6.15]
 [-0.7  -1.4   7.9  -2.9   0.9   0.  ]]
(5, 6)


In [205]:
constr_rhs = coef[1:,-1] #rhs 
print(constr_rhs)

[ 24.8   0.8 -71.4  95.  -47. ]


In [206]:
lower_bound = 0
upper_bound = np.inf

In [207]:
col_indices = np.arange(N)
print(col_indices)

[0 1 2 3 4 5]


In [208]:
#declare the decision variables in the model
model_lab4.x = Var(col_indices)

In [209]:
row_indices = np.arange(M)
print(row_indices)

[0 1 2 3 4]


In [210]:
model_lab4.constraints = ConstraintList()

In [211]:
for i in row_indices:
  model_lab4.constraints.add(summation(constr_coef[i],model_lab4.x) <= constr_rhs[i])

In [212]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model_lab4.x[j].setlb(lower_bound)
  model_lab4.x[j].setub(upper_bound)

In [213]:
# add the model objective
model_lab4.objective = Objective(expr = summation(obj_coef,model_lab4.x), sense=maximize)

In [214]:
model_lab4.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [215]:
!apt-get install -y -qq coinor-cbc

In [216]:
opt_cbc = SolverFactory('cbc')

In [217]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


7)solution
Solver status: warning;
Solver termination condition: unbounded
warning status shows that it does not have any feasible region. so termination condition is unbounded.
because of infinty upper bound of variable we didn't get any feasible resion.so termination condition is unbounded.and we not get solution.

Reset sense

In [218]:
model_lab4.objective.set_sense(minimize)


In [219]:
model_lab4.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals
          5 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [220]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

  - termination condition: unbounded
  - message from solver: <undefined>


Solver status: warning
Solver termination condition: unbounded


In [221]:
model_lab4.objective.set_sense(maximize)

In [222]:
upper_bound = 500
for j in col_indices:
  model_lab4.x[j].setub(upper_bound)

In [223]:
model_lab4.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   500 : False :  True :  Reals
          1 :     0 :  None :   500 : False :  True :  Reals
          2 :     0 :  None :   500 : False :  True :  Reals
          3 :     0 :  None :   500 : False :  True :  Reals
          4 :     0 :  None :   500 : False :  True :  Reals
          5 :     0 :  None :   500 : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.65*x[0] + 0.75*x[1] - 5.85*x[2

In [224]:
result = opt_cbc.solve(model_lab4)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [225]:
print('\nObjective = ', model_lab4.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i+1,'] = ', model_lab4.x[i].value)

print('\nConstraints')
model_lab4.constraints.display()


Objective =  848.9752701

Decision Variables
x[ 1 ] =  16.247154
x[ 2 ] =  500.0
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  0.0
x[ 6 ] =  15.447154

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None :          -8.123577 :  24.8
      2 :  None : 0.7999999999999989 :   0.8
      3 :  None :      -1463.9024386 : -71.4
      4 :  None :         94.9999971 :  95.0
      5 :  None :       -711.3730078 : -47.0


9)solution
Objective =  848.9752701

Decision Variables
x1 =  16.247154;
x2 =  500.0;
x3 =  0.0;
x4 =  0.0;
x5 =  0.0;
x6 =  15.447154
after changing the sense we didn't get freasible region. so using sense maximation and giving upper bound to the variable we get the optimal solution.
constraint 2 and constraint 4 are active and constraint 1 3 and 5 are inactive.

